# Internet checksum

Este notebook explica cómo se calcula el checksum que utilizan las cabeceras de IPv4, ICMPv4, IGMP, UDP o TCP. Se le conoce habitualmente como «checksum de Internet» o *Internet checksum*.

El checksum permite detectar cambios (incluso de un solo bit) en la transmisión del mensaje. Consiste en una suma de 16 en complemento a uno.

## ¿Cómo funciona?

1. Se divide el mensaje (la cabecera en el caso de IP) en palabras de 16 bits.
1. Se suman todas las palabras como enteros de 16 bits (con acarreo).
1. Se toma el complemento a uno del resultado: `~suma & 0xFFFF`, se coloca en el campo de la cabecera y se envía el mensaje.
1. Al recibir el mensaje, se repite la suma (incluyendo el checksum) y el resultado debe ser `0xFFFF`.
